### QUES1-FLAG

In [ ]:
import cv2
import numpy as np

def identify_flag(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Resize the image to a fixed size for consistency (adjust if necessary)
    height, width = 300, 600  # Flag aspect ratio
    image = cv2.resize(image, (width, height))

    # Split the image into top and bottom halves
    top_half = image[:height // 2, :]
    bottom_half = image[height // 2:, :]

    # Compute average color for each half (BGR format)
    top_avg_color = np.mean(top_half, axis=(0, 1))
    bottom_avg_color = np.mean(bottom_half, axis=(0, 1))

    print(f"Top half average color: {top_avg_color}")
    print(f"Bottom half average color: {bottom_avg_color}")

    # Check if top is red and bottom is white (Indonesia flag)
    if is_red(top_avg_color) and is_white(bottom_avg_color):
        return "Indonesia Flag"

    # Check if top is white and bottom is red (Poland flag)
    elif is_white(top_avg_color) and is_red(bottom_avg_color):
        return "Poland Flag"

    # If neither, return unknown
    return "Unknown Flag"

def is_red(color):
    b, g, r = color
    # Red color check (R > 130 and G, B are low)
    return r > 130 and g < 100 and b < 100

def is_white(color):
    b, g, r = color
    # White color check (R, G, B all high)
    return r > 180 and g > 180 and b > 180

# Input image path (change this to the path of your flag image)
input_image_path = r"C:\Users\LENOVO\Downloads\flag.jpg"

try:
    result = identify_flag(input_image_path)
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")


### QUES.2- SSD

In [ ]:
import cv2
import tensorflow as tf

model = tf.saved_model.load('http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz')

video_path = r"C:\Users\LENOVO\Desktop\WhatsApp Video 2024-12-21 at 21.44.13_342a5fef.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Width of video frames
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Height of video frames

out = cv2.VideoWriter(
    'output_ssd.avi',  # Output file name
    cv2.VideoWriter_fourcc(*'XVID'),  # Codec for output video
    fps,  # Frames per second
    (width, height)  # Frame size
)

def preprocess(frame):
    # Resize and normalize frame for SSD input
    input_tensor = tf.image.resize_with_pad(frame, 300, 300)
    input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.uint8)
    return tf.expand_dims(input_tensor, 0)

# Loop through each frame in the video
while cap.isOpened():
    ret, frame = cap.read()  # Read a frame from the video
    if not ret:  # If no frame is read (end of video), break the loop
        break

    # Perform object detection
    input_tensor = preprocess(frame)
    detections = model(input_tensor)  # Model output

    # Draw bounding boxes on the frame
    detection_boxes = detections['detection_boxes'][0].numpy()  # Extract boxes
    detection_classes = detections['detection_classes'][0].numpy()
    detection_scores = detections['detection_scores'][0].numpy()

    for i, box in enumerate(detection_boxes):
        if detection_scores[i] > 0.5:  # Confidence threshold
            h, w, _ = frame.shape
            ymin, xmin, ymax, xmax = box
            start_point = (int(xmin * w), int(ymin * h))
            end_point = (int(xmax * w), int(ymax * h))
            frame = cv2.rectangle(frame, start_point, end_point, (0, 255, 0), 2)
            label = str(detection_classes[i])
            frame = cv2.putText(frame, label, start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Write the annotated frame to the output video
    out.write(frame)

    cv2.imshow('SSD', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


### QUES.3-YOLO

In [ ]:
import cv2  
import torch  

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

video_path = r"C:\Users\LENOVO\Desktop\WhatsApp Video 2024-12-21 at 21.44.13_342a5fef.mp4" # Replace with your video file path
cap = cv2.VideoCapture(video_path)  # Open the video file

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()  # Exit the program if the video cannot be opened

fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Width of video frames
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Height of video frames

out = cv2.VideoWriter(
    'output_yolo.avi',  # Output file name
    cv2.VideoWriter_fourcc(*'XVID'),  # Codec for output video
    fps,  # Frames per second
    (width, height)  # Frame size
)
# Loop through each frame in the video
while cap.isOpened():
    ret, frame = cap.read()  # Read a frame from the video
    if not ret:  # If no frame is read (end of video), break the loop
        break

    results = model(frame)

    result_frame = results.render()[0]  # Annotated frame

    cv2.imshow('YOLOv5', result_frame)

    out.write(result_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  
out.release()  
cv2.destroyAllWindows()  